# Extracting and Visualize Chicago Public Data

## This is a Peer-graded Assignment: Analyze the census, crime, and school data for a given neighborhood or district

### installing all modules

In [2]:
!pip install mysql-connector-python

In [4]:
import mysql.connector as connection
data_db = connection.connect(host = 'localhost', user = 'root', password = '%env', db = 'ibm')
cursor = data_db.cursor()

cursor.execute('show databases')
cursor.fetchall()

[('ibm',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sys',)]

In [5]:
cursor.execute('use ibm')
cursor.execute('show tables')
cursor.fetchall()

[('ChicagoCensusData',), ('ChicagoCrimeData_xsl',), ('ChicagoPublicSchools',)]

1. Find the total number of crimes in the CRIME table

In [6]:
import pandas as pd

# extract all data
cursor.execute("select * from ChicagoCrimeData_xsl")
data = cursor.fetchall()
df = pd.DataFrame(data)

# find rows amount
count_row = df.shape[0]
print(count_row)

533


2. List community areas with per capita income less than 1000

In [7]:
cursor.execute("select community_area_name from ChicagoCensusData where per_capita_income < 11000")
census_data = cursor.fetchall()
df = pd.DataFrame(census_data)
print(df)

                    0
0  West Garfield Park
1      South Lawndale
2         Fuller Park
3           Riverdale


3. List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis

In [9]:
cursor.execute("select case_number from ChicagoCrimeData_xsl where description like '%MINOR%'")
minor_crime_data = cursor.fetchall()
df = pd.DataFrame(minor_crime_data)
print(df)

          0
0  HL266884
1  HK238408


4. List all kidnapping crimes involving a child

In [10]:
cursor.execute("select * from ChicagoCrimeData_xsl where primary_type like '%KIDNAP%' and description like '%CHILD%'")
kidnap_crime_data = cursor.fetchall()
df = pd.DataFrame(kidnap_crime_data, columns=['ID', 'CASE_NUMBER', 'DATE', 'BLOCK', 'IUCR', 'PRIMARY_TYPE', 'DESCRIPTION', 'LOCATION_DESCRIPTION', 'ARREST', 'DOMESTIC', 'BEAT', 'DISTRICT', 'WARD', 'COMMUNITY_AREA_NUMBER', 'FBICODE', 'X_COORDINATE', 'Y_COORDINATE', 'YEAR', 'LATITUDE', 'LONGITUDE', 'LOCATION'])
print(df)

          ID CASE_NUMBER       DATE                 BLOCK    IUCR  \
0  5276766.0    HN144152 2007-01-26  050XX W VAN BUREN ST  1792.0   

  PRIMARY_TYPE               DESCRIPTION LOCATION_DESCRIPTION ARREST DOMESTIC  \
0   KIDNAPPING  CHILD ABDUCTION/STRANGER               STREET  False    False   

   ... DISTRICT  WARD COMMUNITY_AREA_NUMBER FBICODE X_COORDINATE Y_COORDINATE  \
0  ...     15.0  29.0                  25.0    20.0    1143050.0    1897546.0   

     YEAR            LATITUDE           LONGITUDE  \
0  2007.0  41.874908410000000  -87.75024931000000   

                        LOCATION  
0  (41.874908413, -87.750249307)  

[1 rows x 21 columns]


5. What kind of crimes were recorded at school

In [11]:
cursor.execute("select distinct primary_type from ChicagoCrimeData_xsl where location_description like '%SCHOOL%'")
school_crime_data = cursor.fetchall()
df = pd.DataFrame(school_crime_data)
print(df)

                        0
0                 BATTERY
1         CRIMINAL DAMAGE
2               NARCOTICS
3                 ASSAULT
4       CRIMINAL TRESPASS
5  PUBLIC PEACE VIOLATION


6. List the average safety score for all type of schools

In [12]:
cursor.execute("select Elementary_Middle_or_High_School, AVG(safety_score) from ChicagoPublicSchools group by Elementary_Middle_or_High_School")
school_safety_score = cursor.fetchall()
df = pd.DataFrame(school_safety_score)
print(df)

    0        1
0  ES  49.5204
1  HS  49.6235
2  MS  48.0000


7. List 5 community areas with highest % of households below poverty line

In [13]:
cursor.execute("select community_area_name, percent_households_below_poverty from ChicagoCensusData order by  percent_households_below_poverty desc limit 5")
community_data = cursor.fetchall()
df = pd.DataFrame(community_data)
print(df)

                    0     1
0           Riverdale  56.5
1         Fuller Park  51.2
2           Englewood  46.6
3      North Lawndale  43.1
4  East Garfield Park  42.4


8. Which community area(number) is most crime prone

In [14]:
cursor.execute("select community_area_number, count(*) as frequency from ChicagoCrimeData_xsl group by  community_area_number order by frequency desc limit 1")
community_number_data = cursor.fetchall()
df = pd.DataFrame(community_number_data)
print(df)

      0   1
0  25.0  43


9. Use a sub-query to find the name of the community area with the highest hardship index

In [15]:
cursor.execute("select community_area_name from ChicagoCensusData where hardship_index = (select max(hardship_index) from ChicagoCensusData)")
hard_index_data = cursor.fetchall()
df = pd.DataFrame(hard_index_data)
print(df)

           0
0  Riverdale


10. Use a sub-query to determine the Community Area Name with most number of crimes

In [16]:
cursor.execute("select community_area_name from ChicagoCensusData where community_area_number = (select community_area_number from ChicagoCrimeData_xsl where community_area_number = 25 limit 1)")
community_area_data = cursor.fetchall()
df = pd.DataFrame(community_data)
print(df)

                    0     1
0           Riverdale  56.5
1         Fuller Park  51.2
2           Englewood  46.6
3      North Lawndale  43.1
4  East Garfield Park  42.4
